In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [151]:
df = pd.read_csv('car.csv')
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
df.columns = columns

In [152]:
df.head()


,buying,maint,doors,persons,lug_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [153]:
df.shape

(1728, 7)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   buying      1728 non-null   object
 1   maint       1728 non-null   object
 2   doors       1728 non-null   object
 3   persons     1728 non-null   object
 4   lug_boot    1728 non-null   object
 5   safety      1728 non-null   object
 6   evaluation  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [155]:
# df['Person_Capacity'] = LE.fit_transform(df['Person_Capacity'])
import warnings
warnings.filterwarnings("ignore")
mapping = {
    
    'm_buying_and_maint' : {
        # For 'buying' & 'maint' columns.
        'low': 1,
        'med': 2,
        'high': 3,
        'vhigh': 4,
    },
    
    'm_doors' : {
        # For 'doors' columns.
        '2': 1,
        '3': 2,
        '4': 3,
        '5more': 4,
    },
    
    'm_persons' : {
        # For 'persons' columns.
        '2': 1,
        '4': 2,
        'more': 3,
    },
    
    'm_lug_boot' : {
        # For 'lug_boot' column.
        'small': 1,
        'med': 2,
        'big': 3,
    },
    
    'm_safety' : {
        # For 'safety' column.
        'low': 1,
        'med': 2,
        'high': 3,
    },
    
    'm_evaluation' : {
        # For 'evaluation' column.
        'unacc': 1,
        'acc': 2,
        'good': 3,
        'vgood': 4
    }
}

df_encoded = df

df_encoded['buying'] = df['buying'].replace(mapping['m_buying_and_maint'])
df_encoded['maint'] = df['maint'].replace(mapping['m_buying_and_maint'])
df_encoded['doors'] = df['doors'].replace(mapping['m_doors'])
df_encoded['persons'] = df['persons'].replace(mapping['m_persons'])
df_encoded['lug_boot'] = df['lug_boot'].replace(mapping['m_lug_boot'])
df_encoded['safety'] = df['safety'].replace(mapping['m_safety'])
df_encoded['evaluation'] = df['evaluation'].replace(mapping['m_evaluation'])

df_encoded.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,4,4,1,1,1,1,1
1,4,4,1,1,1,2,1
2,4,4,1,1,1,3,1
3,4,4,1,1,2,1,1
4,4,4,1,1,2,2,1


In [156]:
df.dtypes

buying        int64
maint         int64
doors         int64
persons       int64
lug_boot      int64
safety        int64
evaluation    int64
dtype: object

In [157]:
# df = df[(df['evaluation'] == 1) | (df['evaluation'] == 2)]

In [158]:
X = df.iloc[:,:-1]
X

,buying,maint,doors,persons,lug_boot,safety
0,4,4,1,1,1,1
1,4,4,1,1,1,2
2,4,4,1,1,1,3
3,4,4,1,1,2,1
4,4,4,1,1,2,2
...,...,...,...,...,...,...
1723,1,1,4,3,2,2
1724,1,1,4,3,2,3
1725,1,1,4,3,3,1
1726,1,1,4,3,3,2


In [159]:
y = df.iloc[:,-1:]
y

,evaluation
0,1
1,1
2,1
3,1
4,1
...,...
1723,3
1724,4
1725,1
1726,3


In [160]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [161]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
RF = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
from hyperopt import hp,tpe,fmin,STATUS_OK,Trials

In [162]:
RF = RandomForestClassifier(criterion = "gini", max_features =  "sqrt", n_estimators =   100)

In [163]:
RF.fit(X_train,y_train)


RandomForestClassifier()

In [164]:
y_hat = RF.predict(X_test)


In [165]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test,y_hat))


0.9739884393063584


In [186]:
n = 100
RF.predict([X_test.iloc[n]]), X_test.iloc[n].values, y_test.iloc[n]

(array([1], dtype=int64),
 array([4, 4, 3, 2, 3, 2], dtype=int64),
 evaluation    1
 Name: 70, dtype: int64)

In [178]:
RF.predict_proba([X_test.iloc[n]])

array([[0.01, 0.18, 0.72, 0.09]])

In [167]:
print(classification_report(y_test,y_hat))


              precision    recall  f1-score   support

           1       0.99      1.00      1.00       235
           2       0.97      0.93      0.95        83
           3       0.71      0.91      0.80        11
           4       0.94      0.88      0.91        17

    accuracy                           0.97       346
   macro avg       0.90      0.93      0.91       346
weighted avg       0.98      0.97      0.97       346



In [168]:
print(confusion_matrix(y_test,y_hat))


[[235   0   0   0]
 [  2  77   4   0]
 [  0   0  10   1]
 [  0   2   0  15]]


In [169]:
RF

RandomForestClassifier()

In [174]:
import pickle
with open('RF.pkl', 'wb') as model_file:
    pickle.dump(RF, model_file)

In [175]:
RF.n_features_in_

6

In [176]:
import joblib
import onnx
import skl2onnx
from skl2onnx.convert import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load the .pkl model
model = joblib.load('RF.pkl')

# Convert the model to ONNX
onnx_model = convert_sklearn(model, initial_types=[('input', FloatTensorType([None, model.n_features_in_]))])

# Save the ONNX model
onnx.save_model(onnx_model, 'RF.onnx')


In [177]:
import onnx

# Load the ONNX model
model = onnx.load("RF.onnx")

# Print input names
print("Input names:")
for input in model.graph.input:
    print(input.name)

# Print output names
print("Output names:")
for output in model.graph.output:
    print(output.name)


Input names:
input
Output names:
output_label
output_probability


In [108]:
import onnx

# Load the ONNX model
model_path = "DT.onnx"  # Update with your actual model path
model = onnx.load(model_path)

# Get the model's input names
print("Input names:")
for input in model.graph.input:
    print(input.name)

# Get the model's output names
print("\nOutput names:")
for output in model.graph.output:
    print(output.name)


Input names:
input

Output names:
output_label
output_probability


In [180]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("RF.onnx")
inputs = {session.get_inputs()[0].name: np.array([[3,2,1,2,1,3]], dtype=np.float32)}
outputs = session.run(None, inputs)
print(outputs)


[array([2], dtype=int64), [{1: 0.12999999523162842, 2: 0.8399998545646667, 3: 0.029999999329447746, 4: 0.0}]]


In [ ]:
scp -P 2222 -r /mnt/c/Users/A507658/Downloads/qwen2-docker/RF.onnx root@localhost:/root/